In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import datetime
from pyspark.sql import Window
from pyspark.sql.types import *
import sys
import os
app_name = 'selection bu batch100'
# spark = SparkSession.builder.appName(app_name).enableHiveSupport().getOrCreate()

#os.environ['PYSPARK_PYTHON'] = 'python3.5'

os.environ['PYSPARK_PYTHON'] = "/usr/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/usr/bin/python3"

spark = (SparkSession \
         .builder \
         .appName("spark_test") \
         .enableHiveSupport() \
         .config("spark.executor.instances", "100") \
         .config("spark.executor.memory", "48g") \
         .config("spark.executor.cores", "48") \
         .config("spark.driver.memory", "48g") \
         .config("spark.sql.shuffle.partitions", "200000") \
         .config("spark.default.parallelism", "200000") \
         .config("spark.driver.maxResultSize", "8g") \
         .config("spark.pyspark.python", "/usr/bin/python3")\
         .config('spark.yarn.driver.memoryOverhead', "64g")
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.executorEnv.yarn.nodemanager.container-executor.class","DockerLinuxContainer")\
         .config("spark.yarn.appMasterEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .config("spark.executorEnv.yarn.nodemanager.docker-container-executor.image-name","bdp-docker.jd.com:5000/wise_mart_rmb_py36:latest")\
         .getOrCreate())

spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

import datetime
import sys
import pandas as pd
import os
import numpy as np
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
import pyspark.sql.types as sql_type
import spa_utils

spark.sql("set hive.exec.dynamic.partition=true")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.sql('''set hive.exec.max.dynamic.partitions=200551''')
spark.sql('''set hive.exec.max.dynamic.partitions.pernode=200551''')
params = {'author':'xiaoxiao10'}

In [2]:
data = spark.sql('''select * from app.app_pa_price_baseprice_self_nonpromo_flag_60_9 ''')

# data = data.filter(F.col('item_sku_id')=='100000197925')
data = data.select(['item_sku_id','sale_qtty', 'price', 'baseprice','uv','stock_status' ,'non_promo_flag', 'dt'])

In [3]:
inputSchema = StructType([StructField('item_sku_id',sql_type.StringType()),StructField('sale_qtty',sql_type.FloatType()),
                           StructField('price',sql_type.FloatType()),StructField('baseprice',sql_type.FloatType()),
                           StructField('uv',sql_type.IntegerType()),StructField('stock_status',sql_type.FloatType()),
                          StructField('non_promo_flag',sql_type.IntegerType()),StructField('base_qtty',sql_type.FloatType()),
                           StructField('dt',sql_type.StringType())])

def map_func(dictionary):
    return(str(dictionary['item_sku_id']),
    float(dictionary['sale_qtty']),
    float(dictionary['price']),
    float(dictionary['baseprice']),
    int(dictionary['uv']),
    float(dictionary['stock_status']),
    int(dictionary['non_promo_flag']),
    float(dictionary['base_qtty']),
    str(dictionary['dt']))

def cal(x,df):
    rate = 0.03
    x_series = x.iloc[0]
    df_1 = df.loc[(abs(df['price']-x_series['baseprice'])<rate*x_series['baseprice'])&(df['dt']!=x_series['dt'])&(df['stock_status']>=0.15)]
    df_1['index_2'] = abs(df_1['index']-x_series['index'])
    try:
        if (min(df_1['index_2']) > 30):
            df_2 = df_1.loc[df_1['index_2']<=60]
        else:
            df_2 = df_1.loc[df_1['index_2']<=30]        

        df_2_index = list(df_2.sort_values(['index_2','index']).head(6)['index'])

        a = df_2.loc[df_2['index'].isin(df_2_index)]
        a['weight'] = np.log(abs(a['uv']-x_series['uv'])+1)
        a['weight'] = a['weight'].map(lambda x:0.3 if x==0 else x)
        a['weight'] = 1/a['weight']
        smooth_qtty_1 = sum(a['weight']*a['sale_qtty'])/sum(a['weight'])      
    except:
        smooth_qtty_1 = -9999
    x['base_qtty'] = smooth_qtty_1
    return x

def smooth(row):    
    df= pd.DataFrame(list(row[1]),columns=['item_sku_id','sale_qtty','price','baseprice','uv','stock_status','non_promo_flag', 'dt'])
    df = df.sort_values('dt').reset_index(drop=True)
    df['index'] = df['dt'].map(lambda x:(datetime.datetime.strptime(x,'%Y-%m-%d')-datetime.datetime.strptime(min(df['dt']),'%Y-%m-%d')).days)
    df['base_qtty'] = -9999
    
    result = df.groupby('dt').apply(lambda x:cal(x,df))
    result = result[['item_sku_id','sale_qtty','price','baseprice','uv','stock_status','non_promo_flag','base_qtty','dt']]
    
    return result.to_dict(orient= 'records')

spark.sql('''set hive.exec.dynamic.partition=true''')
spark.sql('''set hive.exec.dynamic.partition.mode=nostrict''')
spark.sql('''set hive.exec.max.dynamic.partitions=200551''')
spark.sql('''set hive.exec.max.dynamic.partitions.pernode=200551''')
spark.sql('''set hive.exec.max.created.files=20055100''')
spark.sql('''set mapreduce.job.split.metainfo.maxsize=-1''')
spark.sql('''set yarn.app.mapreduce.am.command-opts=-Xmx3072m''')
spark.sql('''set yarn.app.mapreduce.am.resource.mb=4096''')
spark.sql("set hive.exec.dynamic.partition=true")
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")

DataFrame[key: string, value: string]

## 插入新的日期的数据

In [4]:
origin_day = spark.sql('''select max(dt) from app.app_pa_baseline_baseprice_60_xiaoxiao_21''').collect()[0][0]

df_1 = data.rdd.map(lambda row: (row['item_sku_id'],row)).groupByKey().flatMap(lambda row: smooth(row))
df_final = spark.createDataFrame(df_1.map(map_func),schema=inputSchema)

df_final = df_final.select(['item_sku_id', 'sale_qtty', 'price', 'baseprice','uv','stock_status', 'non_promo_flag','base_qtty', 'dt'])\
                   .filter(F.col('dt')>origin_day)
spa_utils.save_hive_result(df_final,'app.app_pa_baseline_baseprice_60_xiaoxiao_21',partitioning_columns=['dt'],write_mode='insert',spark=spark,params=params)

# 仅show

In [5]:
df_2 = spark.sql('''select * from app.app_pa_baseline_baseprice_60_xiaoxiao_21''')
df_3 = df_2.filter(F.col('base_qtty')!=-9999)

df_3.cache()
df_3.count()

862268631

In [6]:
df_3 = df_3.withColumn('true_flag',F.when(((F.col('price')>F.col('baseprice'))&(F.col('sale_qtty')<F.col('base_qtty')))|
                                          ((F.col('price')<F.col('baseprice'))&(F.col('sale_qtty')>F.col('base_qtty'))),F.lit('true'))\
                                    .when(((F.col('price')>F.col('baseprice'))&(F.col('sale_qtty')>F.col('base_qtty')))|
                                          ((F.col('price')<F.col('baseprice'))&(F.col('sale_qtty')<F.col('base_qtty'))),F.lit('false'))\
                                    .when(((F.col('price')==F.col('baseprice'))&(F.col('sale_qtty')!=F.col('base_qtty')))|
                                          ((F.col('price')!=F.col('baseprice'))&(F.col('sale_qtty')==F.col('base_qtty'))),F.lit('non_equal'))\
                                    .when((F.col('price')==F.col('baseprice'))&(F.col('sale_qtty')==F.col('base_qtty')),F.lit('equal')))

df_3.groupby('true_flag').agg(F.count('dt').alias('count')).show()

+---------+---------+
|true_flag|    count|
+---------+---------+
|     true|227703700|
|    false| 82797464|
|non_equal|221157391|
|    equal|330610076|
+---------+---------+

